## Create a model that is able to predict both Age and gender based on the image presented


In [20]:
import torch
import numpy as np, cv2, pandas as pd, glob, time
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models, datasets
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
import pandas as pd, numpy as np
import os
print(os.getcwd())


/Users/robertocandelario/Documents/Python Projects/Age_Gender_Classification


In [21]:
path = os.path.join(os.getcwd(), "fairface-img-margin025-trainval/")

In [25]:
trn_df = pd.read_csv(path + 'fairface-labels-train.csv')
val_df = pd.read_csv(path + 'fairface-labels-val.csv')
val_df.head()

,file,age,gender,race,service_test
0,val/1.jpg,11,Male,East Asian,False
1,val/2.jpg,51,Female,East Asian,True
2,val/3.jpg,37,Male,White,True
3,val/4.jpg,25,Female,Latino_Hispanic,True
4,val/5.jpg,24,Male,Southeast Asian,False


### Build a subclass Dataset 
    the subclass will take a dataframe and makes the required transformations.


In [31]:
IMAGE_SIZE = 224
class GenderAgeClass(Dataset):
    
    def __init__(self, df, tfms=None):
        self.df = df
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, ix):
        f = self.df.iloc[ix].squeeze()
        file = f.file
        gen = f.gender == 'Female'
        age = f.age
        im = cv2.imread(file)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        return im, age, gen
    
    def preprocess_image(self, im):
        im = cv2.resize(im, (IMAGE_SIZE, IMAGE_SIZE))
        im = torch.tensor(im).permute(2,0,1)
        im = self.normalize(im/225.)
        return im[None]
    
    def collate_fn(self, batch):
        'preprocess images, ages and genders'
        ims, ages, genders = [], [], []
        for im, age, gender in batch:
            im = self.preprocess_image(im)
            ims.append(im)
            
            ages.append(float(int(age)/80))
            genders.append(float(gender))
            
        ages, genders = [torch.tensor(x).to(device).float() for x in [ages, gender]]
        ims = torch.cat(ims).to(device)
            
        return ims, ages, genders

## Define the dataloaders
    The dataloaders help pull batches of data for the model to be trained on


In [32]:
trn = GenderAgeClass(trn_df)
val = GenderAgeClass(val_df)

train_loader = DataLoader(trn, batch_size=32, shuffle=True,
                        drop_last=True,collate_fn=trn.collate_fn)
test_loader = DataLoader(val, batch_size=32,
                        collate_fn=val.collate_fn)

a,b,c, = next(iter(train_loader))
print(a.shape, b.shape, c.shape)

torch.Size([32, 3, 224, 224]) torch.Size([32]) torch.Size([])


## Get the pretrained model
    Free the current parameters and change the last layer 

In [33]:
def get_model():
    model = models.vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
        
    model.avgpool = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten()
    )
    ### Build a nn.Module class. This class will help us split the model to predict classification and sequential 
    
    class ageGenderClassifier(nn.Module):
        def __init__(self):
            super(ageGenderClassifier, self).__init__()
            
            # First layer 
            self.intermediate = nn.Sequential(
                nn.Linear(2048, 512),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Dropout(0.4),
                nn.Linear(128,64),
                nn.ReLU()
            )
            
            # Next two layers are the ending layers 
            self.age_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
            
            self.gender_classifier = nn.Sequential(
                nn.Linear(64, 1),
                nn.Sigmoid()
            )
        
        def forward(self, x):
            x = self.intermediate(x)
            age = self.age_classifier(x)
            gender = self.gender_classifier(x)
            return gender, age
        
    model.classifier = ageGenderClassifier()
    
    # define the loss function for both gender and age 
    gender_criterion = nn.BCELoss()
    age_criterion = nn.L1Loss()
    loss_functions = gender_criterion, age_criterion
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    return model.to(device), loss_functions, optimizer

model, criterion, optimizer = get_model()

## Define function to train model on a batch

In [35]:
def train_batch(data, model, optimizer, criteria):
    model.train()
    ims, age, gender = data
    optimizer.zero_grad()
    pred_gender, pred_age = model(ims)
    gender_criterion, age_criterion = criteria
    gender_loss = gender_criterion(pred_gender.squeeze(), gender)
    age_loss = age_criterion(pred_age.squeeze(), gender)
    
    # add both losses and perform back propagation
    total_loss = gender_loss + age_loss
    total_loss.backward()
    optimizer.step()
    return total_loss